## Audio objects

Audio data are at the core or pyfar and three classes can be used for storing, processing, and visualizing such data. There are three different audio objects

- `pyfar.Signal` can be used to store equidistant and complete signals that can be converted between the time and frequency domain.
- `pyfar.TimeData` and `pyfar.FrequencyData` are intended data that can not be converted between the time and frequency domain. Examples for this are time data with missing data, frequency data that is only available at certain frequencies, or an energy histogram.

The following examples introduce fundamental concepts behind audio classes and show how the data inside audio objects can be accessed. See the `pyfar documentation` for a complete overview.

In [1]:
import pyfar as pf

### Creating audio objects

Creating an audio object is similar across classes. To create a `TimeData` object, specify the data and the times at which the data was sampled.

In [2]:
# time data with non-equidistant sampling times
time = pf.TimeData([1, 0, -1], [0, 1, 3])

Creating a `FrequencyData` object requires the specification of the frequencies that belong to the data

In [3]:
# band limited frequency data object
frequency = pf.FrequencyData([1, .8, .7], [400, 800, 1600])

A `Signal` object can be created in the time or frequency data and requires the sampling rate of the signal

In [4]:
# create signal object with time data and a sampling rate of 4 Hz
signal = pf.Signal([1, 0, 0, 0], 4)

# creating a signal object in the frequency domain requires additional
# parameters
signal2 = pf.Signal([1, 1, 1], 4, n_samples=4, domain='freq')

### Accessing data in audio objects

The remainder of the examples focuses on Signal objects as the most common audio type, but there are many analogies between Signal and Time/FrequencyData objects. So using other kinds of audio objects will be fairly simple after understanding Signal objects.

The data stored in audio objects can be accessed trough their `time` and `freq` properties.

In [5]:
signal.time

array([[1., 0., 0., 0.]])

returns the time data and

In [6]:
signal2.freq

array([[1.+0.j, 1.+0.j, 1.+0.j]])

returns the frequency data. There are a few important things to note right away.

- for convenience, the audio data is stored in `numpy` arrays
- data inside audio objects is at least 2-dimensional. In this example `signal` is an audio object with a single channel of audio data and the shape of `signal.time` is `(1, 4)`. In pyfar the samples of time data (4 in this example) and the frequency bins of frequency data are always stored in the last dimension of `signal.time` and `signal.freq`

Intuitively, the `time` property can be used to access data of `TimeData` objects and the `freq` property to access data of `FrequencyData` objects. For `Signal` objects time and frequency data can be accessed at any time and the Signal class converts between the time and frequency domain if required using the Discrete Fourier Transform. 

In [7]:
# Accessing frequency data from the signal that was created with time data
signal.freq

array([[1.+0.j, 1.+0.j, 1.+0.j]])

At this point it is **important to note** that the data returned by the `freq` property can depend on the normalization of the Discrete Fourier Transform as explained in more detail `here`. The frequency data without any normalization can be accessed using the `freq_raw` property. However, for this example the two return the same results

In [8]:
signal.freq_raw

array([[1.+0.j, 1.+0.j, 1.+0.j]])

read on to learn how to find out which normalization of the Discrete Fourier Transform is used for a signal.

### Setting data in audio objects

The same properties can be used to set the data inside audio objects.

In [9]:
signal.time = [2, 0, 0, 0]
print(f'signal.time = {signal.time}')
signal2.freq = [2, 2, 2]
print(f'signal2.freq = {signal2.freq}')

signal.time = [[2. 0. 0. 0.]]
signal2.freq = [[2.+0.j 2.+0.j 2.+0.j]]


### Additional information stored in audio objects

Converting between time and frequency data is already useful, but audio objects can do more. They contain additional information about their data that can be helpful. Some basic information is already returned when prompting an audio object

In [10]:
signal

time domain energy Signal:
(1,) channels with 4 samples @ 4 Hz sampling rate and none FFT normalization

Lets break down the information one by one. 

**Domain:** The signal is in the time domain, because we last accessed its time data above. We can get and set this property with the `signal.domain` attribute but it is of not much interested when working with Signal objects because they are automatically converted between the time and frequency domain, when it is required.

In [11]:
signal.domain

'time'

**Signal Type:** It is an *energy* Signal, which is important to note and understand. Energy signals are finite signals with finite energy. Examples for these kinds of signals are impulse responses. *Power* signals on the other hand have an infinite duration and energy. Examples are noise or sine signals of which we typically observe a block of finite size. The signal type is a read only property that is set by the DFT normalization introduced below.

In [12]:
signal.signal_type

'energy'

**DFT Normlaization:** For *power* Signals, different DFT normalization can be used that differently scale the values stored in `signal.freq`. For *energy* signals, the DFT normalization is always 'none'. For more information refer the `Fast Fourier Transform` examples.

In [13]:
signal.fft_norm

'none'

### Signal shape, length, and axis

Signals can contain multiple channels of audio data and the shape of the data inside the audio object is often important. There multiple attributes describing this.

**Channel shape:** The *channel shape*, in short `cshape` gives the shape of the data inside an audio object but ignores the number of samples or frequency bins. For example our first Signal created above has one channel and 4 samples, hence the data is of chape `(1, 4)`. Because the `cshape` ignores the samples it is `(1, )`. There are two ideas behind this. First, digital signal processing methods often work on channels, and second the length of the signal might be different in the time and frequency domain.

In [14]:
signal.cshape

(1,)

**Length:** The signal length in samples and bins is stored in the properties

In [15]:
signal.n_samples

4

and

In [16]:
signal.n_bins

3

Note that only the half sided spectrum is stored for real valued time signals. Hence the length differs in the time and frequency domain in this case.

**Channel axis:** Some functions in pyfar operate along one or more axes of the data. In analogy, to the channel chape, these axes are referred to as *channel axis* or in short `caxis`. Consider a signal of `cshape=(3, 4)` and a duration of `n_samples=128`. In this case `caxis=0` referst to the first dimension of size 3 and `caxis=-1` referns to the last dimension of size 4 but *not* dimension containing the 128 samples.

### Multidimensional audio objects

Audio objects may contain data in multiple channels and pyfar offers some functionality to ease handling such signals. Let the following signal of `cshape=(2, 3)` symbolize data obtained for t2 loudspeaker and 3 microphone positions

In [17]:
signal3 = pf.Signal(
    [[[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]],
     [[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]]],
    6)

signal3

time domain energy Signal:
(2, 3) channels with 6 samples @ 6 Hz sampling rate and none FFT normalization

A subset of the channels can be accessed through slicing. For example the first channel in both dimensions can be obtained by

In [18]:
channel = signal3[0, 0]

channel

time domain energy Signal:
(1,) channels with 6 samples @ 6 Hz sampling rate and none FFT normalization

as the output tells us this returns a signal object, which is often useful because most pyfar functions require signal objects as input. This also works for obtaining larger slices of the signal, for example

In [19]:
channels = signal3[0]
channels

time domain energy Signal:
(3,) channels with 6 samples @ 6 Hz sampling rate and none FFT normalization

### Further reading

After getting used to adding and accessing data in audio objects, you are ready for discovering ways of inspecting and working with audio objects. Good next steps might be learning how to graphically display (plot) data from audio objects as detailed `here` and check out how to apply simple arithmetic operations with audio objects as described `here`.